# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install py4j

# For maps
!pip install folium
!pip install plotly

Define the environment (Java & Spark homes)

---

In [ ]:
!ls /content

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Starting Spark Session and print the version


---


In [ ]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4500") \
        .getOrCreate()

spark.version

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4500 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Descargar Datasets

In [ ]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/frankenstein.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/el_quijote.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!ls /dataset

# RDD

---



## Ejemplo 1

In [ ]:
textFile = spark.sparkContext.textFile("/dataset/frankenstein.txt")
textFile.first()

Abre la Spark UI e investiga qué ha sucedido


---




## Creación de colecciones paralelizadas
Una manera muy rápida de crear RDD desde la shell, cuando estamos aprendiendo, es crear una colección paralelizada. Para ello:

## Ejemplo 2

In [ ]:
distData = spark.sparkContext.parallelize([25, 20, 15, 10, 5])
distData.reduce(lambda x ,y: x + y)

¿De qué tipo es la variable distData?


## Ejercicio 1
Cuenta el número de líneas del fichero "el_quijote.txt"

---



## Ejercicio 2
Imprime la primera línea del fichero "el_quijote.txt"

---



## Transformaciones y Acciones sobre RDDs 

### Acciones

### Ejemplo 3

In [ ]:
print(textFile.count()) # Número de elementos en el RDD
print(textFile.first()) # Primer elemento del RDD

### Transformaciones

### Ejemplo 4

In [ ]:
# ReduceByKey
lines = spark.sparkContext.textFile("/dataset/frankenstein.txt")
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b).cache()
counts.count()
counts.collect()

In [ ]:
# SortByKey
sorted = counts.sortByKey()
sorted.collect()

Abre la Spark UI e investiga que ha ido pasando.

### Ejemplo 5

In [ ]:
# Filter

linesWithSpark = textFile.filter(lambda line: "the" in line)
linesWithSpark.count()

### Ejercicio 3
Obtén el número de ocurrencias de cada palabra del fichero "frankenstein.txt"

### Ejercicio 4
Obtén el top-10 de palabras con más de 4 caracteres

---



## Key/Value Pair RDD

---



### Ejemplo 6


---



In [ ]:
charac_sw = spark.sparkContext.textFile("/dataset/characters.csv")
planets_sw = spark.sparkContext.textFile("/dataset/planets.csv")
charac_sw.take(10)

In [ ]:
planets_sw.take(10)

In [ ]:
from itertools import islice

charac_sw_noheader = charac_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

planets_sw_noheader = planets_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

### Ejercicio 5
Obtén un listado con la población del planeta al que pertenece cada personaje de Star Wars


---
